In [6]:
%%capture
!rm -r /root/.kaggle
!mkdir /root/.kaggle
!touch /root/.kaggle/kaggle.json

api_token = "TOKEN"

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json
!pip install --upgrade --force-reinstall --no-deps kaggle

# !kaggle competitions download -c birdclef-2022
# !unzip "/content/birdclef-2022.zip"
!kaggle kernels output pjmathematician/autocast-data 

In [7]:
import pandas as pd
dataset = pd.read_csv('/content/all_ac_questions.csv')
dataset.head()

,question,id,background,publish_time,close_time,tags,source_links,prediction_count,forecaster_count,answer,choices,status,qtype,crowd
0,What will the end-of-day closing value for the...,G1,Outcome will be determined by the end-of-day c...,2015-09-01 13:49:29.860000+00:00,2016-01-01 17:00:01+00:00,"['Finance', 'Economic Indicators']",['http://ftalphaville.ft.com/2015/08/17/213732...,1549.0,385,D,"['Less than 6.30', 'Between 6.30 and 6.35, inc...",Resolved,mc,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
1,How many seats will the Justice and Developmen...,G2,The Justice and Development Party (AKP) failed...,2015-09-01 13:54:25.050000+00:00,2015-11-01 22:00:20+00:00,"['Elections and Referenda', 'Non-US Politics']",['http://www.al-monitor.com/pulse/originals/20...,567.0,194,A,"['A majority', 'A plurality', 'Not a plurality']",Resolved,mc,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
2,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30.138000+00:00,2015-11-30 14:00:15+00:00,['Finance'],['http://atimes.com/2015/11/china-will-allow-s...,545.0,148,yes,"['yes', 'no']",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
3,Will the Export-Import Bank of the United Stat...,G5,The Export-Import Bank's authorization expired...,2015-09-01 14:02:21.242000+00:00,2015-12-04 14:00:25+00:00,"['Economic Policy', 'US Politics', 'US Policy']",['http://thehill.com/policy/finance/260118-wee...,1000.0,379,yes,"['yes', 'no']",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
4,Will a trilateral meeting take place between C...,G6,"A trilateral meeting of leaders from China, Ja...",2015-09-01 14:04:41.470000+00:00,2015-12-31 23:00:11+00:00,['Foreign Policy'],"['https://en.wikipedia.org/wiki/Li_Keqiang', '...",946.0,385,no,"['yes', 'no']",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."


In [8]:
features = ['question','background']
label = ['answer']

In [9]:
dataset= dataset[dataset['qtype'] == 't/f'][features + label]
dataset['answer'] = dataset['answer'].str.replace('no','0').str.replace('yes', '1')
dataset['text'] =  dataset['background'] + '\n' + dataset['question']
dataset['label'] = dataset['answer']
dataset = dataset.dropna().reset_index(drop = True)[['text','label']]
dataset['label'] = dataset['label'].astype(int)
dataset

,text,label
0,China suspended initial public offerings (IPOs...,1
1,The Export-Import Bank's authorization expired...,1
2,"A trilateral meeting of leaders from China, Ja...",0
3,For details of the case involving Jason Rezaia...,1
4,A launch for military or testing purposes woul...,1
...,...,...
1950,"Nationwide, new COVID-19 hospitalizations have...",1
1951,"Nationwide, new COVID-19 hospitalizations have...",0
1952,"On February 24, 2022, Russian President Vladim...",0
1953,"A previous Metaculus question asked ""at least ...",1


In [15]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
vec = CountVectorizer()
X = vec.fit_transform(dataset['text'])
y = dataset['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf_xgb = XGBClassifier()
clf_rfc = RandomForestClassifier()
clf_xgb.fit(X_train, y_train)
clf_rfc.fit(X_train, y_train)

In [16]:
print("XGB:",score(y_test, clf_xgb.predict(X_test)))
print("RFC:",score(y_test, clf_rfc.predict(X_test)))

XGB: 0.27621483375959077
RFC: 0.30434782608695654
